# 將Bitstream File 燒錄至板子上

In [37]:
from pynq import Overlay
Design = Overlay("bitstream/CNN.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


# 從硬體檔案中讀取這個系統中的 AXI ADDRESS

In [38]:
Design.ip_dict

{'memory_ctrl_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'memory_ctrl_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_memory_ctrl_0_S00_AXI_reg',
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:user:memory_ctrl:1.0'}}

## 取出 ADDER 這個 AXI IP 的 Base Address 以及 Address Range

##### 可以檢查這個地址是否和Vivado中 Address Editor 內 ADDER 的 Base Address 相同

In [39]:
CNN_ADDR = Design.ip_dict['memory_ctrl_0']['phys_addr']
CNN_ADDR_range = Design.ip_dict['memory_ctrl_0']['addr_range']
print("The Base Address of adder is : " + format(CNN_ADDR, '02x'))
print("The Address Range of adder is : " + format(CNN_ADDR_range, '02x'))

The Base Address of adder is : 43c00000
The Address Range of adder is : 10000


## Import libraries

In [40]:
from PIL import Image
import math
import numpy as np
import cv2

## 使用Pynq 內建的 MMIO 以達成 PS 和 AXI IP 之間的溝通


##### MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

In [41]:
from pynq import MMIO
cnn = MMIO(CNN_ADDR, CNN_ADDR_range)

In [42]:
def write_data_to_feature(addr,data):
    
    cnn.write(0x4,2)
    cnn.write(0x8,data)
    cnn.write(0xc,addr)
    cnn.write(0x0,1)
    cnn.write(0x0,0)
    answer=0
    
    while (answer != 1):
        answer=cnn.read(0x0)
    
    

In [43]:
def write_data_to_weight(addr,data):
    
    cnn.write(0x14,2)
    cnn.write(0x18,data)
    cnn.write(0x1c,addr)
    cnn.write(0x10,1)
    cnn.write(0x10,0)
    answer=0
    
    while (answer != 1):
        answer=cnn.read(0x10)
    

In [44]:
def read_data_from_feature(addr):
    
    cnn.write(0x4,1)
    cnn.write(0xc,addr)
    cnn.write(0x0,1)
    cnn.write(0x0,0)
    answer=0
    
    while (answer != 1):
        answer=cnn.read(0x0)
    
    data=cnn.read(0x4)
    return data
    

In [45]:
def read_data_from_weight(addr):
    
    cnn.write(0x14,1)
    cnn.write(0x1c,addr)
    cnn.write(0x10,1)
    cnn.write(0x10,0)
    answer=0
    
    while (answer != 1):
        answer=cnn.read(0x10)
    
    data=cnn.read(0x14)
    return data
        
    

In [46]:
def hex2dec(string_num):
    return str(int(string_num.upper(), 16)) 

## 輸入input

In [47]:
img = cv2.imread('images/1.png', 0)

h = img.shape[0]
w = img.shape[1]
img = img/255
img = np.round(img*math.pow(2,12))
img = img.astype(np.int64)
img = np.reshape(img,(h*w))
img = img.tolist()
for i in range(len(img)):
    pixel = int(img[i])
    write_data_to_weight(i,pixel)


## 輸入Weight & Bias 

In [48]:
f = open('conv_w+b&fc_w.dat')    #開啟資料檔案檔案
lines = f.readlines()      #把全部資料檔案讀到一個列表lines中
a=[]         
for line in lines:   #把lines中的資料逐行讀取出來
    para = line.strip('\n')  #處理逐行資料：strip表示把頭尾的'\n'去掉
    dec_para = int(hex2dec(para))
    a.append(dec_para)
    
for i in range(len(a)):
    write_data_to_weight(i+784,a[i])
    



## 輸入指令，並做輸入指令的前處理

In [49]:
conv_input_size = 28
kernel_size = 5
kernel_num = 6
stride = 1
operation=1
instruction =  conv_input_size + (kernel_size<<7) + (kernel_num<<10) + (stride<<14) + (operation<<17)
write_data_to_feature(4095,instruction)


In [50]:
pooling_input_size = 24
pooling_size = 2
pooling_kernel_num = 6
operation=2
instruction = pooling_input_size + (pooling_size<<10) + (pooling_kernel_num<<13) + (operation<<17)
write_data_to_feature(4094,instruction)

In [51]:
feature_num = 6 
fc_input_size= 12
class_num = 10
operation=3
instruction = class_num + (fc_input_size<<8) + (feature_num<<12) + (operation<<17)
write_data_to_feature(4093,instruction)

In [52]:
operation=4
instruction = (operation<<17)
write_data_to_feature(4092,instruction)
cnn.write(0x4,3)


In [ ]:
done = 0
while(done != 1):
    done = cnn.read(0x20)

for i in range(10):
    ans = read_data_from_feature(i+1000)
    print(ans)
    
